In [50]:
# Import thư viện

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
import csv

print('- Finish importing package')

- Finish importing package


In [51]:
# Task 1: Tự động đăng nhập vào Linkedin
# Tự động mở trình duyệt Chrome và Login vào Linkedin
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/home' # Gán đường link vào biến url
driver.get(url)
sleep(1)

# Lưu thông tin đăng nhập vào một biến mới nhằm bảo mật thông tin
# Lưu ý phải lưu file txt trên cùng file chương trình
login_input = open('input.txt')
line = login_input.readlines()
username = line[0]
password = line[1]

# Định vị thành phần trên file html bằng .find_element (bằng id)
email_field = driver.find_element(By.ID,'session_key')
# Sau khi xác định được id dùng .send_keys để truyền ký tự muốn nhập vào thành phần
email_field.send_keys(username)
sleep(1)
# Làm tương tự đối với mật khẩu
password_field = driver.find_element(By.NAME, 'session_password') # Cũng có thể tìm kiếm bằng thẻ name
password_field.send_keys(password)
sleep(1)
# Tiếp tục định vị thành phần nút Login in (Bằng cách tìm thành phần bằng Xpath)
login_button = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/form/div[2]/button')
# Sau đó sử dụng phương thức .click()
login_button.click()
sleep(1)

In [52]:
# Task 2: Tìm các user mà bạn muốn Crawl

# Định vị Search Bar (sử dụng Xpath nếu không định vị được id và name)
search_bar_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
sleep(1)
# Nhập từ khoá bạn muốn tìm kiếm lên Search Bar
# search_keyword = input("Nhập thông tin tìm kiếm: ") 
# Chương trình sẽ tạm dừng để bạn nhập keyword
search_bar_field.send_keys('Data Analyst') # Thay vào đây search_keyword
sleep(1)
# Tìm kiếm (không có nút để bấm tìm kiếm --> Ra lệnh nhấn nút ENTER) bằng Keys.RETURN
search_bar_field.send_keys(Keys.RETURN)
print('- Finish Task 2: Search for profiles')

- Finish Task 2: Search for profiles


In [53]:
# Task 3: Scrape the URLs of the profiles
import re
# Chỉ tìm kiếm company
company_search = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[9]/button'))
)
company_search.click()
sleep(2)
# Xây dựng hàm để get URL
def get_url():
    page_source = BeautifulSoup(driver.page_source)
    # Tìm toàn bộ company (các tag 'a' để thu hẹp phạm vi tìm kiếm truyền thêm tham số class)
    profiles = page_source.find_all('a', class_ = 'app-aware-link') # Phải là class_
    # Tạo một List rỗng 
    profile_url_container = []
    # Xây dựng vòng lặp for
    for profile in profiles:
        profile_URL = profile.get('href') # Company profile thường được chứa trong thẻ href
        # Thường đường dẫn của 1 profile không chỉ nằm chìm ở tên mà còn ở profile avt nên cần dùng thêm hàm if
        if re.match(r'^https://www\.linkedin\.com/company/', profile_URL)and profile_URL not in profile_url_container:
            profile_url_container.append(profile_URL)
    return profile_url_container # Function này mỗi khi gọi sẽ trả về 1 list các đường dẫn đến các profiles ở trên được gọi

# Task 4: Thực hiện chuyển trang và get url với từng trang
# Xây dựng hàm chuyển trang và get url

page_num = 2 # Lấy thử trong 3 trang   # Có thể thay thế int(input('Số trang bạn muốn scrape: '))
url_all_page = [] # tạo một List rỗng
for page in range(page_num):
    url_one_page = get_url()
    sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #window.scrollTo(X,Y) X: di chuyển phương ngang, Y di chuyển phương dọc
    sleep(2)
    # Định vị phím next page bằng CLASS_NAME
    next_button = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__button--next'))
    ) # Chờ cho button xuất hiện trong vòng 10 giây để hoàn tất load webpage
    driver.execute_script("arguments[0].click();", next_button)
    url_all_page = url_all_page + url_one_page
    sleep(1)


In [56]:
# Task 5: Crawl thông tin trên từng profile và lưu thành file csv
# Truy cập vào profile đầu tiên trong danh sách đã thu thập 
with open('output.csv', 'w',  newline = '') as file_output: # Mở file CSV
    headers = ['Name', 'Field','URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader() # sử dụng phương thích dict writer cho phép tạo ra một bộ từ điển trong đó tiêu đề là key còn giá trị tương ứng thu thập được là value 
    for linkedin_URL in url_all_page:
        driver.get(linkedin_URL)
        sleep(2)
        print('- Accessing profile: ', linkedin_URL)
        page_source = BeautifulSoup(driver.page_source, 'html.parser') # đại diện cho 1 thư viện mã nguồn mở sử dụng để chia cấu trúc của thư mục html thành nhiều dạng cấu trúc dữ liệu cây mà mình mong muốn
        info_div = page_source.find('div', class_='block mt2')
        # Định vị tên Công ty 
        name = info_div.find('h1').get_text().strip()
        print('---- Company Name is: ', name)
        info_loc = info_div.find_all('div')
        # Định vị lĩnh vực hoạt động
        field = info_loc[1].find('div').get_text().strip()
        print('---- Company Field is: ', field)
        # Ghi dữ liệu vào file CSV trong vòng lặp
        writer.writerow({headers[0]:name, headers[1]:field ,headers[2]:linkedin_URL})
        print('\n') # phương thức writerow tiếp nhận tham số có kiểu dict

- Accessing profile:  https://www.linkedin.com/company/data-analysts/
---- Company Name is:  Data Analyst
---- Company Field is:  Data Infrastructure and Analytics


- Accessing profile:  https://www.linkedin.com/company/freelance-data-analyst/
---- Company Name is:  Data Analyst
---- Company Field is:  Investment Management


- Accessing profile:  https://www.linkedin.com/company/dataanalyst2/
---- Company Name is:  Data Analyst
---- Company Field is:  Advertising Services


- Accessing profile:  https://www.linkedin.com/company/data-analyst/
---- Company Name is:  Data Analyst
---- Company Field is:  Software Development


- Accessing profile:  https://www.linkedin.com/company/galaxy-survey-engineering/
---- Company Name is:  Data Analyst
---- Company Field is:  Civil Engineering


- Accessing profile:  https://www.linkedin.com/company/dataanalystfaycelmaalaoui/
---- Company Name is:  Data Analyst
---- Company Field is:  Research


- Accessing profile:  https://www.linkedin.com/compa